In [70]:
from collections import deque
from builtins import object
import fnmatch
import re

nil = object() # used to distinguish from None

class TrieNode(object):
    """
    Node of trie/Aho-Corasick automaton
    """

    __slots__ = ['char', 'output', 'fail', 'children']

    def __init__(self, char):
            """
            Constructs an empty node
            """

            self.char = char # character
            self.output = nil # an output function for this node
            self.fail = nil # fail link used by Aho-Corasick automaton
            self.children = {} # children

    def __repr__(self):
        """
        Textual representation of node.
        """

        if self.output is not nil:
            return "<TrieNode '%s' '%s'>" % (self.char, self.output)
        else:
            return "<TrieNode '%s'>" % self.char


class Trie(object):
    """
    Trie/Aho-Corasick automaton.
    """

    def __init__(self, case_insensitive = False, wildcards = False):
        """
        Construct an empty trie
        """

        self.root = TrieNode('')
        self.case_insensitive = case_insensitive
        self.wildcards = wildcards

  
    # def __get_node(self, word):
    #     """
    #     Private function retrieving a final node of trie
    #     for given word
    #     Returns node or None, if the trie doesn't contain the word.
    #     """

    #     node = self.root
    #     for c in word:
    #         try:
    #             node = node.children[c]
    #         except KeyError:
    #             try:
    #               node = node.children['*']
    #             except KeyError:
    #               return None

    #     return node


    # def get(self, word, default=nil):
    #     """
    #     Retrieves output value associated with word.
    #     If there is no word returns default value,
    #     and if default is not given rises KeyError.
    #     """
    #     node = self.__get_node(word)
    #     output = nil
    #     if node:
    #         output = node.output

    #     if output is nil:
    #         if default is nil:
    #             raise KeyError("no key '%s'" % word)
    #         else:
    #             return default
    #     else:
    #         return output


    def keys(self):
        """
        Generator returning all keys (i.e. word) stored in trie
        """

        for key, _ in self.items():
            yield key


    def values(self):
        """
        Generator returning all values associated with words stored in a trie.
        """

        for _, value in self.items():
            yield value


    def items(self):
        """
        Generator returning all keys and values stored in a trie.
        """

        L = []
        def aux(node, s):
            s = s + node.char
            if node.output is not nil:
                L.append((s, node.output))

            for child in node.children.values():
                if child is not node:
                    aux(child, s)

        aux(self.root, '')
        return iter(L)


    def __len__(self):
        """
        Calculates number of words in a trie.
        """

        stack = deque()
        stack.append(self.root)
        n = 0
        while stack:
            node = stack.pop()
            if node.output is not nil:
                n += 1

            for child in node.children.values():
                stack.append(child)

        return n


    def add_word(self, word, value):
        """
        Adds word and associated value.
        If word already exists, its value is replaced.
        """
        if not word:
            return
            
            
        if self.case_insensitive:
            word = word.lower()

        node = self.root
        for c in word:
            try:
                node = node.children[c]
            except KeyError:
                n = TrieNode(c)
                node.children[c] = n
                node = n

        node.output = value


    def clear(self):
        """
        Clears trie.
        """

        self.root = TrieNode('')


    def exists(self, word):
        """
        Checks if whole word is present in the trie.
        """

        node = self.__get_node(word)
        if node:
            return bool(node.output != nil)
        else:
            return False


    def match(self, word):
        """
        Checks if word is a prefix of any existing word in the trie.
        """

        return (self.__get_node(word) is not None)


    def make_automaton(self):
        """
        Converts trie to Aho-Corasick automaton.
        """

        queue = deque()

        # 1.
        for i in range(256):
            c = chr(i)
            if c in self.root.children:
                node = self.root.children[c]
                node.fail = self.root # f(s) = 0
                queue.append(node)
            else:
                self.root.children[c] = self.root

        # 2.
        while queue:
            r = queue.popleft()
            for node in r.children.values():
                queue.append(node)
                state = r.fail
                while node.char not in state.children:
                        state = state.fail

                node.fail = state.children.get(node.char, self.root)

# Iter is what we're using to search, will return all words at a given position so if you have
# the word "hers" in your text at position (index) 3 and you're looking for "he" and "hers" 
# (aka they're in your keyword list) then this will return something like [3, ("he", "hers")]
    def iter(self, string):
        """
        Generator performs Aho-Corasick search string algorithm, yielding
        tuples containing two values:
        - position in string
        - outputs associated with matched strings
        """
        state = self.root
        #if wildcards:
        # I want to insert somewhere into the below
        # iteration function an line that will check if a word
        # we're searching for contains a "*" character which indicates
        # a wildcard. if the word contains a wildcard it will use
        # filtered = fnmatch.filter(text, 'th*s') or something similar to returnn
        # instances of the word with wildcards in the text
        for index, c in enumerate(string):
            while c not in state.children:
                if '*' in state.children:
                  c = '*'
                  break;
                else:
                  state = state.fail
            
            state = state.children.get(c, self.root)

            tmp = state
            output = []
            while tmp is not nil:
                if tmp.output is not nil:
                    output.append(tmp.output)
                tmp = tmp.fail
                
            if output:
                yield (index, output)



# Iter long basically finds the string at each index but only returns the longest word
# so if you have "hers" and are looking for the words "he" and "hers", iter_long only
# return "hers" and will ignore "he"
    def iter_long(self, string):
        """
        Generator performs a modified Aho-Corasick search string algorithm,
        which maches only the longest word.
        """
        state = self.root
        last  = None

        index = 0
        while index < len(string):
            c = string[index]

            if c not in state.children:
              c = '*'

            if c in state.children:
                state = state.children[c]

                if state.output is not nil:
                    # save the last node on the path
                    last = (state.output, index)

                index += 1
            else:
                if last:
                    # return the saved match
                    yield last

                    # and start over, as we don't want overlapped results
                    # Note: this leads to quadratic complexity in the worst case
                    index = last[1] + 1
                    state = self.root
                    last  = None
                else:
                    # if no output, perform classic Aho-Corasick algorithm
                    while c not in state.children:
                        state = state.fail

        # corner case
        if last:
            yield last

            
# I don't actually know what this is for 
    def find_all(self, string, callback):
        """
        Wrapper on iter method, callback gets an iterator result
        """
        for index, output in self.iter(string):
            callback(index, output)



if __name__ == '__main__':

    def test_func():
        found = []
        not_found = []
        
        # below code can be modified to remove any non A-Z and a-z characters from 
        # text if desired 
        # regex = re.compile('[^a-zA-Z]')
        # df['Text'] = df['Text'].apply(lambda x: regex.sub(" ", x))
        
        words = "won't he BlAh bluh blch sleep help hers his she hi him m*n".split()
        count = {}

        trie = Trie()
        print("[Key Words]")
        for w in words:
            trie.add_word(w, w)
            count[w] = 0
            print(w)

        s = "her BlAh shershi2$0dawon'tmanza his sleep man men min mon dman fgod ugod wgod" #text in which to search
        # works with spaces or all in one go, is it useful to add a line that removes
        # spaces from the text? 

        
        trie.make_automaton()

        print()
        print("Words Found (iter_long): ")
        for res in trie.iter_long(s):
            pos, matches = res
            count[res[0]] += 1
            print(res)
        
        print()
        print("Words Found (iter): ")
        for res in trie.iter(s):
            pos, matches = res
            print(res)
            for fragment in matches:
                found.append(fragment)


        not_found = set(words)-set(found)

        print()
        print("Words not found: ")
        for n in not_found:
            print(n)
        
        print()
        print("[number of mentions]")
        print(count)
  
    test_func()

[Key Words]
won't
he
BlAh
bluh
blch
sleep
help
hers
his
she
hi
him
m*n

Words Found (iter_long): 
('he', 1)
('BlAh', 7)
('she', 11)
("won't", 25)
('m*n', 28)
('his', 34)
('sleep', 40)
('m*n', 44)
('m*n', 48)
('m*n', 52)
('m*n', 56)
('m*n', 61)

Words Found (iter): 
(1, ['he'])
(7, ['BlAh'])
(11, ['she', 'he'])
(13, ['hers'])
(15, ['hi'])
(25, ["won't"])
(28, ['m*n'])
(33, ['hi'])
(34, ['his'])
(40, ['sleep'])
(44, ['m*n'])
(48, ['m*n'])
(52, ['m*n'])
(56, ['m*n'])
(61, ['m*n'])

Words not found: 
help
bluh
him
blch

[number of mentions]
{"won't": 1, 'he': 1, 'BlAh': 1, 'bluh': 0, 'blch': 0, 'sleep': 1, 'help': 0, 'hers': 0, 'his': 1, 'she': 1, 'hi': 0, 'him': 0, 'm*n': 6}
